In [ ]:
!pip install -r requirements.txt
!pip install scikit-image
!pip install opencv_python

: 

## Frame Difference


In [19]:
import cv2
import time
from skimage.metrics import structural_similarity as compare_ssim

fps_limit = 1
threshold = 0.90
font = cv2.FONT_HERSHEY_SIMPLEX
capture= cv2.VideoCapture("/home/sidk_1023/Videos/Webcam/WhatsApp Video 2022-02-20 at 12.46.21.mp4")
frame_number = 0

start_time = 0
ret, current_frame = capture.read()
previous_frame = current_frame


while(True):
    try:
        now_time = time.time()-start_time
        if now_time > 1.0 / fps_limit:
            start_time = time.time()  # reset the time
            current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
            previous_frame_gray = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)    
            (score,frame_diff) = compare_ssim(previous_frame_gray,current_frame_gray,full = True)
            text = str(score)
            
            if (score<threshold):
                cv2.putText(current_frame, 'Change: '+text, (50,50), font, 1, (0, 0, 255), 2)
                print(f'Change at frame: {frame_number} at time {str(capture.get(cv2.CAP_PROP_POS_MSEC))}')
                
            else:
                cv2.putText(current_frame, 'No Change: '+text, (50,50), font, 1, (0, 255, 0), 2)
            
        cv2.imshow('frame diff ',current_frame)  
        previous_frame = current_frame.copy()
        ret, current_frame = capture.read() 
        frame_number+=1          
    except cv2.error:
        break   
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
         
capture.release()
cv2.destroyAllWindows()

Change at frame: 350 at time 11678.333333333334
Change at frame: 543 at time 18118.333333333336
Change at frame: 747 at time 24926.666666666668


## Anti Spoofing

In [1]:
import os
import cv2
import numpy as np
import warnings
import time
from src.anti_spoof_predict import AntiSpoofPredict
from src.generate_patches import CropImage
from src.utility import parse_model_name
warnings.filterwarnings('ignore')

fps_limit = 1
frame_number = 0
start_time = 0

cascPath = '/home/sidk_1023/spoofing_detection/python_scripts/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)
cap = cv2.VideoCapture("/home/sidk_1023/Videos/Webcam/WhatsApp Video 2022-02-20 at 12.46.21.mp4")
model_dir = "./resources/anti_spoof_models"
while(True):
    try:
        model_test = AntiSpoofPredict(0)
        image_cropper = CropImage()
        now_time = time.time()-start_time
        ret, frame = cap.read()
        if now_time > 1.0 / fps_limit:
            start_time = time.time() 
            image_bbox = model_test.get_bbox(frame)
            prediction = np.zeros((1, 3))
            # sum the prediction from single model's result
            for model_name in os.listdir(model_dir):
                h_input, w_input, model_type, scale = parse_model_name(model_name)
                param = {
                    "org_img": frame,
                    "bbox": image_bbox,
                    "scale": scale,
                    "out_w": w_input,
                    "out_h": h_input,
                    "crop": True,
                }
                if scale is None:
                    param["crop"] = False
                img = image_cropper.crop(**param)
                
                prediction += model_test.predict(img, os.path.join(model_dir, model_name))
                

            # draw result of prediction
            label = np.argmax(prediction)
            value = prediction[0][label]
            if label == 1:
                #print("Image '{}' is Real Face. Score: {:.2f}.".format(image_name, value))
                
                result_text = "RealFace Score: {:.2f}".format(value)
                color = (255, 0, 0)
            else:
                #print("Image '{}' is Fake Face. Score: {:.2f}.".format(image_name, value))
                print(f'fake face at frame: {frame_number} at time {str(cap.get(cv2.CAP_PROP_POS_MSEC))}')
                result_text = "FakeFace Score: {:.2f}".format(value)
                color = (0, 0, 255)
            #print("Prediction cost {:.2f} s".format(test_speed))
            cv2.rectangle(
                frame,
                (image_bbox[0], image_bbox[1]),
                (image_bbox[0] + image_bbox[2], image_bbox[1] + image_bbox[3]),
                color, 2)
            cv2.putText(
                frame,
                result_text,
                (image_bbox[0], image_bbox[1] - 5),
                cv2.FONT_HERSHEY_COMPLEX, 0.5*frame.shape[0]/1024, color)

        frame_number+=1
        cv2.imshow("Frame", frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    except cv2.error:
        break 
        

cap.release()
cv2.destroyAllWindows()

fake face at frame: 118 at time 3936.666666666667
fake face at frame: 288 at time 9610.000000000002
fake face at frame: 313 at time 10445.0
fake face at frame: 338 at time 11278.333333333334
fake face at frame: 363 at time 12113.333333333334
fake face at frame: 412 at time 13748.333333333336
fake face at frame: 460 at time 15350.000000000002
fake face at frame: 485 at time 16183.333333333334
fake face at frame: 510 at time 17018.333333333336
fake face at frame: 535 at time 17851.666666666668
fake face at frame: 559 at time 18653.333333333336


In [7]:
!pip install torch